# Cleaning the sampled dataset

For introduction and sampling of the dataset visit [this](nyc-collisions-data-sampling.ipynb) page.  

Imports and constants:

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

data_path = "data/nyc_sample.csv"

The dataset is in a CSV file called nyc_sample.csv. Let's read the data into a pandas dataframe and inspect the first few rows of the data:

In [26]:
nyc_data = pd.read_csv(data_path)
nyc_data.head(5)

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,08/19/2018,0:40,NaN,NaN,40.622414,-73.896484,"(40.622414, -73.896484)",BELT PARKWAY,NaN,NaN,...,Unspecified,NaN,NaN,NaN,3963695,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,03/08/2018,13:00,QUEENS,11432.0,40.720165,-73.799020,"(40.720165, -73.79902)",168 STREET,GOETHALS AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,3859433,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
2,11/11/2018,15:40,BRONX,10465.0,40.827282,-73.830170,"(40.827282, -73.83017)",HUNTINGTON AVENUE,LAFAYETTE AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4028471,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
3,11/16/2018,22:00,NaN,NaN,40.699936,-73.911810,"(40.699936, -73.91181)",WYCKOFF AVENUE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4028778,Dump,Sedan,NaN,NaN,NaN
4,01/03/2018,13:05,BRONX,10458.0,40.855690,-73.881035,"(40.85569, -73.881035)",NaN,NaN,2475 SOUTHERN BOULEVARD,...,Driver Inattention/Distraction,NaN,NaN,NaN,3821653,PASSENGER VEHICLE,Van,NaN,NaN,NaN


First, before cleaning the data, let's check the column names again and rename some of them, while make them uniform.

In [27]:
nyc_data.columns

Index(['DATE', 'TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'UNIQUE KEY', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5'],
      dtype='object')

While cleaning the column names, let's use list comprehension to chain the vectorised operations on strings without having to type .str every time.

In [33]:
to_be_replaced = {" ": "_", "number_of_": "", "_name": "", "_code": "", "contributing_factor": "cause"}

nyc_data.columns = [col.lower().replace("", "") for col in nyc_data.columns]
# nyc_data.columns = [col.lower().replace(" ", "_").replace("number_of_", "").replace("_name", "").replace("_code", "").replace("contributing_factor", "cause") for col in nyc_data.columns]

In [34]:
nyc_data.columns

Index(['date', 'time', 'borough', 'zip code', 'latitude', 'longitude',
       'location', 'on street name', 'cross street name', 'off street name',
       'number of persons injured', 'number of persons killed',
       'number of pedestrians injured', 'number of pedestrians killed',
       'number of cyclist injured', 'number of cyclist killed',
       'number of motorist injured', 'number of motorist killed',
       'contributing factor vehicle 1', 'contributing factor vehicle 2',
       'contributing factor vehicle 3', 'contributing factor vehicle 4',
       'contributing factor vehicle 5', 'unique key', 'vehicle type code 1',
       'vehicle type code 2', 'vehicle type code 3', 'vehicle type code 4',
       'vehicle type code 5'],
      dtype='object')

Our column names look much better, but let's change their order and simplify some of them. 

In [35]:
new_columns = ['unique_key', 'date', 'time', 'borough', 'zip_code', 'latitude', 'longitude', 'location', 
               'on_street', 'cross_street', 'off_street',
               'pedestrians_injured', 'cyclist_injured', 'motorist_injured', 'total_injured', 
               'pedestrians_killed', 'cyclist_killed', 'motorist_killed', 'total_killed', 
               'vehicle_type_1', 'vehicle_type_2', 'vehicle_type_3', 'vehicle_type_4', 'vehicle_type_5', 
               'cause_vehicle_1', 'cause_vehicle_2','cause_vehicle_3', 'cause_vehicle_4', 'cause_vehicle_5'
              ]
nyc_data.columns = new_columns

In [36]:
nyc_data.columns

Index(['unique_key', 'date', 'time', 'borough', 'zip_code', 'latitude',
       'longitude', 'location', 'on_street', 'cross_street', 'off_street',
       'pedestrians_injured', 'cyclist_injured', 'motorist_injured',
       'total_injured', 'pedestrians_killed', 'cyclist_killed',
       'motorist_killed', 'total_killed', 'vehicle_type_1', 'vehicle_type_2',
       'vehicle_type_3', 'vehicle_type_4', 'vehicle_type_5', 'cause_vehicle_1',
       'cause_vehicle_2', 'cause_vehicle_3', 'cause_vehicle_4',
       'cause_vehicle_5'],
      dtype='object')